# Computational Photography : Image Denoising and Inpainting
___
_Computational photography or computational imaging refers to digital image capture and processing techniques that use digital computation instead of optical processes. Computational photography can improve the capabilities of a camera, or introduce features that were not possible  with film based photography_

## Image Denoising
In earlier examples, we have seen many image smoothing techniques like Gaussian Blurring, Median Blurring etc and they were good to some extent in removing small quantities of noise. In those techniques, we took a small neighbourhood around a pixel and did some operations like gaussian weighted average, median of the values etc to replace the central element. In short, noise removal at a pixel was local to its neighbourhood.

Here we look at the Non-local Means Denoising algorithm to remove noise in the image and learn about functions included in openCV that can be directly used for Image Denoising.

OpenCV provides four variations of this technique.

* <a href="http://docs.opencv.org/3.0.0/d1/d79/group__photo__denoise.html#ga76abf348c234cecd0faf3c42ef3dc715">cv2.fastNlMeansDenoising()</a> - works with a single grayscale images

* <a href="http://docs.opencv.org/3.0.0/d1/d79/group__photo__denoise.html#ga21abc1c8b0e15f78cd3eff672cb6c476">cv2.fastNlMeansDenoisingColored()</a> - works with a color image.

* <a href="http://docs.opencv.org/3.0.0/d1/d79/group__photo__denoise.html#gaf4421bf068c4d632ea7f0aa38e0bf172">cv2.fastNlMeansDenoisingMulti()</a> - works with image sequence captured in short period of time (grayscale images)

* <a href="http://docs.opencv.org/3.0.0/d1/d79/group__photo__denoise.html#gaa501e71f52fb2dc17ff8ca5e7d2d3619">cv2.fastNlMeansDenoisingColoredMulti()</a> - same as above, but for color images.

Common arguments are:

* h : parameter deciding filter strength. Higher h value removes noise better, but removes details of image also. (10 is ok)
* hForColorComponents : same as h, but for color images only. (normally same as h)
* templateWindowSize : should be odd. (recommended 7)
* searchWindowSize : should be odd. (recommended 21)

___
Here we will look at examples for colored images and image sequences:

<a href="http://docs.opencv.org/3.0.0/d1/d79/group__photo__denoise.html#ga21abc1c8b0e15f78cd3eff672cb6c476"> 2. cv2.fastNlMeansDenoisingColored()</a> : 

As mentioned above it is used to remove noise from color images. (Noise is expected to be gaussian). See the example below:

In [8]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
img = cv2.imread('images/noise1.jpg')
dst = cv2.fastNlMeansDenoisingColored(img,None,10,10,7,21)
plt.subplot(121),plt.imshow(img)
plt.subplot(122),plt.imshow(dst)
plt.show()

After de-noising, the resulting image would look like:
<img src="captures/denoise1.png">
___

<a href="http://docs.opencv.org/3.0.0/d1/d79/group__photo__denoise.html#gaf4421bf068c4d632ea7f0aa38e0bf172">2. cv2.fastNlMeansDenoisingMulti()</a>

Now we will apply the same method to a video. The first argument is the list of noisy frames. Second argument imgToDenoiseIndex specifies which frame we need to denoise, for that we pass the index of frame in our input list. Third is the temporalWindowSize which specifies the number of nearby frames to be used for denoising. It should be odd. In that case, a total of temporalWindowSize frames are used where central frame is the frame to be denoised. For example, you passed a list of 5 frames as input. Let imgToDenoiseIndex = 2 and temporalWindowSize = 3. Then frame-1, frame-2 and frame-3 are used to denoise frame-2. Let's see an example.

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
cap = cv2.VideoCapture('vtest.avi')
# create a list of first 5 frames
img = [cap.read()[1] for i in xrange(5)]
# convert all to grayscale
gray = [cv2.cvtColor(i, cv2.COLOR_BGR2GRAY) for i in img]
# convert all to float64
gray = [np.float64(i) for i in gray]
# create a noise of variance 25
noise = np.random.randn(*gray[1].shape)*10
# Add this noise to images
noisy = [i+noise for i in gray]
# Convert back to uint8
noisy = [np.uint8(np.clip(i,0,255)) for i in noisy]
# Denoise 3rd frame considering all the 5 frames
dst = cv2.fastNlMeansDenoisingMulti(noisy, 2, 5, None, 4, 7, 35)
plt.subplot(131),plt.imshow(gray[2],'gray')
plt.subplot(132),plt.imshow(noisy[2],'gray')
plt.subplot(133),plt.imshow(dst,'gray')
plt.show()

It takes considerable amount of time for computation. In the result, first image is the original frame, second is the
noisy one, third is the denoised image.

___

## Image Inpainting

Inpainting is used to remove small noises, strokes etc in old photographs. Almost everyone has encountered old degraded photos with some black spots, some strokes etc on it. How can we restore them back? We can’t simply erase them in a paint tool because it is will simply replace black structures with white structures which is of no use. In these cases, a technique called image inpainting is used.

In the digital world, inpainting (also known as image interpolation or video interpolation) refers to the application of sophisticated algorithms to replace lost or corrupted parts of the image data (mainly small regions or to remove small defects)

The basic idea is simple: Replace those bad marks with its neighbouring pixels so that it looks like the neigbourhood.

We need to create a mask of same size as that of input image, where non-zero pixels corresponds to the area which
is to be inpainted. Everything else is simple. My image is degraded with some black strokes (I added manually). I
created a corresponding strokes with Paint tool.

In [1]:
import numpy as np
import cv2
img = cv2.imread('messi_2.jpg')
mask = cv2.imread('mask2.png',0)
dst = cv2.inpaint(img,mask,3,cv2.INPAINT_TELEA)
dst2 = cv2.inpaint(img,mask,3,cv2.INPAINT_NS)
cv2.imshow('dst',dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

The resulting output would look like this. First image shows degraded input. Second image is the mask. Third and the Fourth images are the result of the following two inpainting algorithms pre-defined in OpenCV:
* INPAINT_TELEA Method by Alexandru Telea <a href = "http://docs.opencv.org/3.0.0/d0/de3/citelist.html#CITEREF_Telea04">[119
]</a> 
* INPAINT_NS Navier-Stokes based method [Navier01]

The function _cv2.inpaint()_ reconstructs the selected image area from the pixel near the area boundary. The function may be used to remove dust and scratches from a scanned photo, or to remove undesirable objects from still images or video.

<img src="images/inpaint_result.jpg">
___